# --- Day 1: Sonar Sweep ---

### --- Part One ---

In [1]:
with open("day1-input1.txt", 'r') as file:
    inputs = [int(line.replace('\n', '')) for line in file.readlines()]

In [2]:
inputs[-5:]

[8299, 8306, 8317, 8320, 8341]

In [3]:
counter = 0
for i in range(1, len(inputs)):
    if inputs[i] > inputs[i-1]:
        counter += 1

In [4]:
counter

1581

### --- Part Two ---

In [5]:
counter = 0
for i in range(1, len(inputs)-2):
    A = sum(inputs[i-1:i+2])
    B = sum(inputs[i:i+3])
    if B > A:
        counter += 1

In [6]:
counter

1618

# --- Day 2: Dive! ---

### --- Part One ---

In [7]:
with open("day2-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [8]:
depth = 0
horizontal = 0

for direction, v in [move.split() for move in inputs]:
    if direction == "forward":
        horizontal += int(v)
    elif direction == "down":
        depth += int(v)
    elif direction == "up":
        depth -= int(v)

In [9]:
depth * horizontal

1507611

### --- Part Two ---

In [10]:
depth = 0
horizontal = 0
aim = 0

for direction, v in [move.split() for move in inputs]:
    if direction == "forward":
        horizontal += int(v)
        depth += aim * int(v)
    elif direction == "down":
        aim += int(v)
    elif direction == "up":
        aim -= int(v)

In [11]:
depth * horizontal

1880593125

# --- Day 3: Binary Diagnostic ---

### --- Part One ---

In [12]:
with open("day3-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [13]:
cols_num = len(inputs[0])
rows_num = len(inputs)
print("Cols:", cols_num, "Rows:", rows_num)


Cols: 12 Rows: 1000


In [14]:
import numpy as np

diagnostic_data = np.asarray([int(cell) for row in inputs for cell in row]).reshape(1000,12)

In [15]:
diagnostic_data[:5]

array([[0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0]])

In [16]:
gamma = ''
for i in range(12):
    gamma += '1' if diagnostic_data[:,i].sum() > rows_num/2 else '0'
epsilon = ''.join(['0' if v == '1' else '1' for v in gamma]) 
print("  gamma: ", gamma)
print("epsilon: ", epsilon)

  gamma:  011100101100
epsilon:  100011010011


In [17]:
int(gamma, 2) * int(epsilon, 2)

4147524

### --- Part Two ---

In [18]:
def rating_calc(data, col=0, mode='oxygen'):
    if len(data) == 1:
        return data[0]
    # for oxygen:
    keep_val =  1 if data[:,col].sum() >= len(data)/2 else 0
    if mode == 'co2':
        keep_val =  1 - keep_val
    data = data[data[:,col] == keep_val]
    return rating_calc(data, col+1, mode)

In [19]:
oxygen_rating = rating_calc(diagnostic_data, mode='oxygen')
co2_rating = rating_calc(diagnostic_data, mode='co2')
print(oxygen_rating, co2_rating)

[0 1 0 1 1 0 0 1 0 0 1 1] [1 0 0 1 1 1 0 0 0 1 1 0]


In [20]:
# convert rating to binary string and then convert to int
get_rating = lambda x: int(''.join([str(v) for v in x]), 2)

In [21]:
get_rating(oxygen_rating) * get_rating(co2_rating) 

3570354

# --- Day 4: Giant Squid ---

### --- Part One ---

In [22]:
with open("day4-numbers.txt", 'r') as file:
    numbers = file.read()

In [23]:
numbers = [int(number) for number in numbers.split(',')]

In [24]:
with open("day4-boards.txt", 'r') as file:
    boards_raw = file.read().split('\n\n')

In [25]:
boards = [list(map(int, board.replace('\n', ' ').replace('  ', ' ').strip().split(' ')))
          for board in boards_raw]

In [26]:
def check_bingo(boards):
    pattern = np.asarray([True,True,True,True,True])
    horizontal = np.where((boards==pattern).all(axis=2))[0]
    vertical = np.where((boards==pattern.reshape(5,1)).all(axis=1))[0]
    return np.append(horizontal, vertical)

In [27]:
def calc_score(bingo, boards_np, boards_np_01, last_number):
    winning_board_01 = boards_np_01[bingo]
    winning_board_01_inv = np.logical_not(winning_board_01)
    winning_board = boards_np[bingo]
    final_score = winning_board[winning_board_01_inv].sum() * last_number
    return final_score

In [28]:
import numpy as np

def play_bingo(boards, get_best=True):
    boards_np = np.asarray(boards).reshape(-1,5,5)
    boards_np_01 = np.zeros_like(boards_np)
    for i in numbers:
        boards_np_01 += boards_np == i
        bingo = check_bingo(boards_np_01)
        if len(bingo) > 0:
            last_board_score = calc_score(bingo, boards_np, boards_np_01, i)
            if get_best:
                print("BINGO!!!")
                break;
            # set values to something that will prevent these boards from further processing
            boards_np_01[bingo] = 0
            boards_np[bingo] = -1
    return last_board_score

In [29]:
play_bingo(boards)

BINGO!!!


31424

### --- Part Two ---

In [30]:
play_bingo(boards, get_best=False)

23042

# --- Day 5: Hydrothermal Venture ---

### --- Part One ---

In [31]:
with open("day5-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]
    inputs = [line.replace(' -> ', ',').split(',') for line in inputs]

In [32]:
# select just horizontal and vertical lines
inputs_hor_and_vert = [coords for coords in inputs
                      if coords[0]==coords[2] or coords[1]==coords[3]]

In [33]:
import itertools

def get_range(v1, v2):
    pts_range = list(range(min(v1, v2), max(v1, v2) + 1))
    if v1 > v2:
        # range needs to be in correct order (important for diagonals)
        pts_range = list(reversed(pts_range))
    return pts_range

def line2pts(x1, y1, x2, y2):
    x_range = get_range(x1, x2)
    y_range = get_range(y1, y2)

    if len(x_range) != len(y_range):
        # this is for horizontal and vertical lines
        pts = list(itertools.product(x_range, y_range))
    else:
    # for 45 degrees diagonals
        pts = zip(x_range, y_range)

    return pts

In [34]:
import collections

def calculate_danger_pts(inputs):
    pts_list = []
    for coords in inputs:
        coords = [int(coord) for coord in coords]
        pts_list += line2pts(*coords)

    duplicate_pts = [
        item for item, count in
        collections.Counter(pts_list).items()
        if count > 1
    ]
    return len(duplicate_pts)

In [35]:
calculate_danger_pts(inputs_hor_and_vert)

5197

### --- Part Two ---

In [36]:
calculate_danger_pts(inputs)

18605

# --- Day 6: Lanternfish ---

### --- Part One ---

In [37]:
with open("day6-input1.txt", 'r') as file:
    inputs = file.read()
    # ex: '[1,2,3,4,..]'
    inputs = [int(v) for v in inputs.split(',')]
    # ex: [1,2,3,4,..]

In [38]:
import numpy as np

def fish_spawner(input_fish, days):
    timers_np = np.asarray(inputs)

    for d in range(days):
        spawn_mask = timers_np == 0
        timers_np -= 1
        timers_np[spawn_mask] = 6
        timers_np = np.append(
            timers_np, 
            np.full(spawn_mask.sum(), 8))
    return len(timers_np)

In [39]:
fish_spawner(inputs, 80)

394994

### --- Part Two ---

In [40]:
import collections
import itertools

def fish_spawner_scalable(inputs, days):
    # turn inputs into dict of counts
    timer_dict = dict(itertools.product(range(9), [0]))
    counter = collections.Counter(inputs)
    for i, v in counter.items():
        timer_dict[i] += v
    # ex: { 0: 218, 1: 50, 2: 30, ..., 8: 0}
        
    for _ in range(days):
        # keep number of newborns
        newborns = timer_dict[0]
        # rotate values in the dict
        for i in range(0,8):
            timer_dict[i] = timer_dict[i+1]
        # add # of fish which spawned new fish to # of fish with timer 6
        timer_dict[6] += newborns
        # assign newly spawned fish to timer 8
        timer_dict[8] = newborns
    return sum(timer_dict.values())

In [41]:
fish_spawner_scalable(inputs, 256)

1765974267455